In [2]:
#!/usr/bin/env python3
"""
MySQL Verbindungstest

Dieses Skript testet die Verbindung zu einer MySQL-Datenbank.
"""

import os
import sys
import mysql.connector
from mysql.connector import Error
from dotenv import load_dotenv

# Versuche, .env-Datei zu laden, falls vorhanden
load_dotenv()

# Datenbank-Konfiguration aus Umgebungsvariablen oder direkt definieren
DB_CONFIG = {
    'host':'db.riespatrick.de',
    'database': os.getenv('DB_NAME', 'your_database'),
    'user': os.getenv('DB_USER', 'your_username'),
    'password': os.getenv('DB_PASSWORD', 'your_password'),
    # Port ist optional, Standard-Port ist 3306
    # 'port': os.getenv('DB_PORT', 3306),
}

def test_connection():
    """Testet die Verbindung zur MySQL-Datenbank."""
    connection = None
    try:
        # Verbindung zur Datenbank herstellen
        connection = mysql.connector.connect(**DB_CONFIG)
        
        if connection.is_connected():
            db_info = connection.get_server_info()
            print(f"✅ Erfolgreich verbunden mit MySQL Server, Version: {db_info}")
            
            # Datenbankname anzeigen
            cursor = connection.cursor()
            cursor.execute("SELECT DATABASE();")
            record = cursor.fetchone()
            print(f"   Verbunden mit Datenbank: {record[0]}")
            
            # Tabellen auflisten
            cursor.execute("SHOW TABLES;")
            tables = cursor.fetchall()
            
            if tables:
                print("\n📋 Verfügbare Tabellen:")
                for table in tables:
                    print(f"   - {table[0]}")
                
                # Optional: Zeige Spalten der ersten Tabelle
                first_table = tables[0][0]
                cursor.execute(f"DESCRIBE {first_table};")
                columns = cursor.fetchall()
                
                print(f"\n📊 Struktur der Tabelle '{first_table}':")
                for column in columns:
                    print(f"   - {column[0]} ({column[1]})")
            else:
                print("   Keine Tabellen in der Datenbank gefunden.")
                
    except Error as e:
        print(f"❌ Fehler bei Verbindung zur MySQL-Datenbank: {e}")
    finally:
        # Verbindung schließen, falls offen
        if connection and connection.is_connected():
            if 'cursor' in locals():
                cursor.close()
            connection.close()
            print("\n🔒 MySQL-Verbindung geschlossen.")

if __name__ == "__main__":
    print("🔄 Teste MySQL-Verbindung...")
    
    # Überprüfen, ob notwendige Umgebungsvariablen gesetzt sind
    missing_vars = []
    for key in ['DB_HOST', 'DB_NAME', 'DB_USER', 'DB_PASSWORD']:
        if not os.getenv(key):
            missing_vars.append(key)
    
    if missing_vars:
        print(f"⚠️  Warnung: Folgende Umgebungsvariablen sind nicht gesetzt: {', '.join(missing_vars)}")
        print("   Verwende Standard-Werte aus dem Skript.")
        
        # Möglichkeit, Werte manuell einzugeben
        use_defaults = input("Möchtest du die Default-Werte verwenden? (j/n): ")
        if use_defaults.lower() != 'j':
            DB_CONFIG['host'] = input("DB_HOST: ") or DB_CONFIG['host']
            DB_CONFIG['database'] = input("DB_NAME: ") or DB_CONFIG['database']
            DB_CONFIG['user'] = input("DB_USER: ") or DB_CONFIG['user']
            DB_CONFIG['password'] = input("DB_PASSWORD: ") or DB_CONFIG['password']
    
    test_connection()

🔄 Teste MySQL-Verbindung...
✅ Erfolgreich verbunden mit MySQL Server, Version: 5.5.5-10.5.28-MariaDB-0+deb11u1
   Verbunden mit Datenbank: dbjobs

📋 Verfügbare Tabellen:
   - pma__bookmark
   - pma__central_columns
   - pma__column_info
   - pma__designer_settings
   - pma__export_templates
   - pma__favorite
   - pma__history
   - pma__navigationhiding
   - pma__pdf_pages
   - pma__recent
   - pma__relation
   - pma__savedsearches
   - pma__table_coords
   - pma__table_info
   - pma__table_uiprefs
   - pma__tracking
   - pma__userconfig
   - pma__usergroups
   - pma__users

📊 Struktur der Tabelle 'pma__bookmark':
   - id (int(10) unsigned)
   - dbase (varchar(255))
   - user (varchar(255))
   - label (varchar(255))
   - query (text)

🔒 MySQL-Verbindung geschlossen.
